<a href="https://colab.research.google.com/github/KellySantosG/Progrmaci-n-tareas-/blob/main/Proyecto_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Kelly Santos García

# Lo que importamos
import numpy as np
import pandas as pd

# Empieza el codigo
# Clase base para los clasificadores
class Clasificadores:
    def __init__(self):
        self.entrenado = False

    def entrenar(self, X, y):
        """
        Método abstracto para entrenar el clasificador.
        """
        raise NotImplementedError("El método 'entrenar' debe ser implementado por las subclases.")

    def predecir(self, X):
        """
        Método abstracto para realizar predicciones.
        """
        if not self.entrenado:
            raise Exception("El clasificador debe estar entrenado antes de predecir.")
        raise NotImplementedError("El método 'predecir' debe ser implementado por las subclases.")

# Implementación de la Regresión Logística
class RegresionLogistica(Clasificadores):
    def __init__(self, tasa_aprendizaje=0.01, iteraciones=1000):
        super().__init__()
        self.tasa_aprendizaje = tasa_aprendizaje
        self.iteraciones = iteraciones
        self.pesos = None

    def _sigmoide(self, z):
        return 1 / (1 + np.exp(-z))

    def entrenar(self, X, y):
        m, n = X.shape
        self.pesos = np.zeros(n)
        for _ in range(self.iteraciones):
            z = np.dot(X, self.pesos)
            predicciones = self._sigmoide(z)
            gradiente = np.dot(X.T, (predicciones - y)) / m
            self.pesos -= self.tasa_aprendizaje * gradiente
        self.entrenado = True

    def predecir(self, X):
        z = np.dot(X, self.pesos)
        return (self._sigmoide(z) >= 0.5).astype(int)

# Implementación de k-Nearest Neighbors
class KNN(Clasificadores):
    def __init__(self, k=3):
        super().__init__()
        self.k = k
        self.X_entrenamiento = None
        self.y_entrenamiento = None

    def entrenar(self, X, y):
        self.X_entrenamiento = X
        self.y_entrenamiento = y
        self.entrenado = True

    def predecir(self, X):
        predicciones = []
        for x in X:
            distancias = np.linalg.norm(self.X_entrenamiento - x, axis=1)
            indices = np.argsort(distancias)[:self.k]
            etiquetas = self.y_entrenamiento[indices]
            predicciones.append(np.argmax(np.bincount(etiquetas)))
        return np.array(predicciones)

# Implementación de SVM
class SVM(Clasificadores):
    def __init__(self, tasa_aprendizaje=0.01, iteraciones=1000, C=1.0):
        super().__init__()
        self.tasa_aprendizaje = tasa_aprendizaje
        self.iteraciones = iteraciones
        self.C = C
        self.pesos = None

    def entrenar(self, X, y):
        m, n = X.shape
        self.pesos = np.zeros(n)
        for _ in range(self.iteraciones):
            for i in range(m):
                if y[i] * np.dot(X[i], self.pesos) < 1:
                    self.pesos += self.tasa_aprendizaje * (y[i] * X[i] + (-2 * (1 / self.C) * self.pesos))
                else:
                    self.pesos += self.tasa_aprendizaje * (-2 * (1 / self.C) * self.pesos)
        self.entrenado = True

    def predecir(self, X):
        return np.sign(np.dot(X, self.pesos))

# Implementación de Árbol de Decisión
class NodoArbol:
    def __init__(self, caracteristica=None, umbral=None, izquierda=None, derecha=None, valor=None):
        self.caracteristica = caracteristica
        self.umbral = umbral
        self.izquierda = izquierda
        self.derecha = derecha
        self.valor = valor

class ArbolDecision(Clasificadores):
    def __init__(self, profundidad_maxima=5):
        super().__init__()
        self.profundidad_maxima = profundidad_maxima
        self.raiz = None

    def _calcular_gini(self, y):
        proporciones = np.bincount(y) / len(y)
        return 1 - np.sum(proporciones**2)

    def _dividir(self, X, y, caracteristica, umbral):
        izquierda = np.where(X[:, caracteristica] <= umbral)
        derecha = np.where(X[:, caracteristica] > umbral)
        return (X[izquierda], y[izquierda]), (X[derecha], y[derecha])

    def _crear_arbol(self, X, y, profundidad):
        if profundidad == self.profundidad_maxima or len(set(y)) == 1:
            valor = np.argmax(np.bincount(y))
            return NodoArbol(valor=valor)

        caracteristica, umbral = None, None
        mejor_gini = float("inf")
        for i in range(X.shape[1]):
            umbrales = np.unique(X[:, i])
            for u in umbrales:
                (X_izq, y_izq), (X_der, y_der) = self._dividir(X, y, i, u)
                gini = (len(y_izq) * self._calcular_gini(y_izq) + len(y_der) * self._calcular_gini(y_der)) / len(y)
                if gini < mejor_gini:
                    mejor_gini, caracteristica, umbral = gini, i, u

        (X_izq, y_izq), (X_der, y_der) = self._dividir(X, y, caracteristica, umbral)
        izquierda = self._crear_arbol(X_izq, y_izq, profundidad + 1)
        derecha = self._crear_arbol(X_der, y_der, profundidad + 1)
        return NodoArbol(caracteristica=caracteristica, umbral=umbral, izquierda=izquierda, derecha=derecha)

    def entrenar(self, X, y):
        self.raiz = self._crear_arbol(X, y, 0)
        self.entrenado = True

    def _predecir_unico(self, nodo, x):
        if nodo.valor is not None:
            return nodo.valor
        if x[nodo.caracteristica] <= nodo.umbral:
            return self._predecir_unico(nodo.izquierda, x)
        return self._predecir_unico(nodo.derecha, x)

    def predecir(self, X):
        return np.array([self._predecir_unico(self.raiz, x) for x in X])

# Implementación de Random Forest
class RandomForest(Clasificadores):
    def __init__(self, n_estimadores=10, profundidad_maxima=5):
        super().__init__()
        self.n_estimadores = n_estimadores
        self.profundidad_maxima = profundidad_maxima
        self.bosque = []

    def entrenar(self, X, y):
        m = len(y)
        for _ in range(self.n_estimadores):
            indices = np.random.choice(m, m, replace=True)
            X_muestra = X[indices]
            y_muestra = y[indices]
            arbol = ArbolDecision(profundidad_maxima=self.profundidad_maxima)
            arbol.entrenar(X_muestra, y_muestra)
            self.bosque.append(arbol)
        self.entrenado = True

    def predecir(self, X):
        predicciones = np.array([arbol.predecir(X) for arbol in self.bosque])
        return np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=predicciones)

# Código principal para prueba de clasificadores (bloque __main__)
if __name__ == "__main__":
    try:
        # Cargar los datos
        data = pd.read_csv('/content/drive/MyDrive/cancer.csv')

        # Separar datos para entrenamiento y prueba
        X = data.iloc[:, 2:].values  # Características
        y = (data['diagnosis'] == 'M').astype(int).values  # Etiquetas binarias (M = 1, B = 0)
        X_entrenamiento, X_prueba = X[:400], X[400:]
        y_entrenamiento, y_prueba = y[:400], y[400:]

        # Crear y entrenar clasificadores
        clasificadores = {
            "Regresión Logística": RegresionLogistica(),
            "KNN": KNN(),
            "SVM": SVM(),
            "Árbol de Decisión": ArbolDecision(),
            "Random Forest": RandomForest()
        }

        for nombre, clasificador in clasificadores.items():
            clasificador.entrenar(X_entrenamiento, y_entrenamiento)
            y_pred = clasificador.predecir(X_prueba)
            precision = np.mean(y_pred == y_prueba)
            print(f"{nombre}: Precisión = {precision:.2f}")

    except Exception as e:
        print(f"Error en la ejecución: {e}")

<ipython-input-10-200404879ad2>:36: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


Regresión Logística: Precisión = 0.94
KNN: Precisión = 0.94
SVM: Precisión = 0.21
Árbol de Decisión: Precisión = 0.93
